In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
# !wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
!pip install sentence-transformers -q

     |████████████████████████████████| 81kB 4.5MB/s 
     |████████████████████████████████| 2.1MB 11.1MB/s 
     |████████████████████████████████| 1.2MB 60.9MB/s 
     |████████████████████████████████| 901kB 54.3MB/s 
     |████████████████████████████████| 3.3MB 61.0MB/s 


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")

In [ ]:
# import sparknlp 
# from sparknlp.pretrained import PretrainedPipeline
# spark = sparknlp.start()

# print("Spark NLP version: {}".format(sparknlp.version()))
# print("Apache Spark version: {}".format(spark.version))

In [ ]:
!wget https://ndownloader.figshare.com/files/16188500 -q

In [ ]:
# We don't need these. I added glove and bert vectors lower down.
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
# !gunzip ./GoogleNews-vectors-negative300.bin.gz

In [ ]:
# from gensim import models
# w = models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
!tar -xvf 16188500

rumoureval2019/
rumoureval2019/final-eval-key.json
rumoureval2019/LICENSE
rumoureval2019/home_scorer_macro.py
rumoureval2019/README
rumoureval2019/rumoureval-2019-training-data.zip
rumoureval2019/rumoureval-2019-test-data.zip


In [ ]:
!unzip -qn rumoureval2019/rumoureval-2019-training-data.zip

In [ ]:
#### IMPORT ####

from pyspark import SparkContext
from pyspark.ml.feature import Tokenizer
# from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, expr, explode, struct, regexp_replace, collect_list, lit
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, Row
# Don't import all here, it overrides the sum function
# from pyspark.sql.functions import *
from pyspark.ml.feature import * 
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import re
from pyspark.ml.linalg import VectorUDT, Vectors
from sentence_transformers import SentenceTransformer, util
from functools import partial

In [ ]:
spark = SparkSession.builder.appName("YourTest").getOrCreate()


In [ ]:
#### DOWNLOAD SOURCE TWEETS && REPLY TWEETS ###
path = "./rumoureval-2019-training-data/twitter-english/*/*/source-tweet/*.json"
source_tweets_df = spark.read.json(path)
path = "./rumoureval-2019-training-data/twitter-english/*/*/replies/*.json"
reply_tweets_df = spark.read.json(path)

In [ ]:
print(reply_tweets_df.schema)
print()

StructType(List(StructField(contributors,StringType,true),StructField(coordinates,StructType(List(StructField(coordinates,ArrayType(DoubleType,true),true),StructField(type,StringType,true))),true),StructField(created_at,StringType,true),StructField(entities,StructType(List(StructField(hashtags,ArrayType(StructType(List(StructField(indices,ArrayType(LongType,true),true),StructField(text,StringType,true))),true),true),StructField(media,ArrayType(StructType(List(StructField(display_url,StringType,true),StructField(expanded_url,StringType,true),StructField(id,LongType,true),StructField(id_str,StringType,true),StructField(indices,ArrayType(LongType,true),true),StructField(media_url,StringType,true),StructField(media_url_https,StringType,true),StructField(sizes,StructType(List(StructField(large,StructType(List(StructField(h,LongType,true),StructField(resize,StringType,true),StructField(w,LongType,true))),true),StructField(medium,StructType(List(StructField(h,LongType,true),StructField(resize

In [ ]:
#### DOWNLOAD TRUE LABELS ###
schema = StructType([StructField("subtaskaenglish", MapType(StringType(), StringType())),StructField("subtaskbenglish", MapType(StringType(), StringType()))])
dev_key = spark.read.schema(schema).option("multiline", "true").json('rumoureval-2019-training-data/dev-key.json')
train_key = spark.read.schema(schema).option("multiline", "true").json('rumoureval-2019-training-data/train-key.json')

In [ ]:
# TRUE LABELS FOR TASK A
dev_key_taskA = dev_key.select(explode(col("subtaskaenglish")))
train_key_taskA = train_key.select(explode(col("subtaskaenglish")))

In [ ]:
dev_key_taskA.take(5)

[Row(key='553556756857884673', value='comment'),
 Row(key='552801161128853504', value='comment'),
 Row(key='758315535570444290', value='comment'),
 Row(key='498334502041956352', value='query'),
 Row(key='758334112662904834', value='deny')]

In [ ]:
# reply_tweets_df.withColumn('used',lit("0")).select('used').take(2)

In [ ]:
# reply_tweets_df.withColumn('set',lit('unknown')).select('set').take(2)

In [ ]:
# reply_tweets_df.join(dev_key_taskA, reply_tweets_df.id_str == dev_key_taskA.key, 'left').drop('key').withColumnRenamed('value', 'label').select('id_str','label').take(5)

In [ ]:
# TRUE LABELS FOR TASK B
dev_key_taskB = dev_key.select(explode(col("subtaskbenglish")))
train_key_taskB = train_key.select(explode(col("subtaskbenglish")))

In [ ]:
# import sparknlp
# spark = SparkSession.builder.appName("cs631").config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.7.5:3.1.1").getOrCreate()
# spark = sparknlp.start()

In [ ]:
### DATA CLEANING  ##
def clean(df):

    def replace_url(text):
        return re.sub(r'https?:\/\/.*[\r\n]*', 'url_url_url', text, flags=re.MULTILINE)

    replace_url_udf = udf(replace_url, StringType())

    df = df.withColumn('cleaned_text', replace_url_udf(col('text')))

    ### REMOVE @
    df = df.withColumn('cleaned_text', regexp_replace(col('cleaned_text'), r'(@([A-Za-z0-9]+))', ''))

    return df


In [ ]:
words_dict = dict(
      has_belief_words = set("assume believe apparent per-haps suspect think thought consider".split()),
      has_report_words = set("evidence source official footage capture assert told claim according".split()),
      has_doubt_words = set("wonder allege unsure guess speculate doubt".split()),
      has_knowledge = set("confirm definitely admit".split()),
      has_denial_words = set("refuse reject rebuff dismiss contradict oppose".split()),
      has_curse_words = set("lol rofl lmfao yeah stfu aha wtf shit".split()),
      # negation_words = set("no not no one nothing never don’t can’t hardly".split(),
      has_question_words = set("when which what who how whom why whose".split()),
      has_other_words = set("irresponsible careless liar false witness untrue neglect integrity murder fake".split())
      # has_false_synonyms = set(['false',  'bogus',  'deceitful',  'dishonest',
      #                 'distorted',  'erroneous',  'fake', 'fanciful',
      #                 'faulty',  'fictitious',  'fraudulent',
      #                 'improper',  'inaccurate',  'incorrect',
      #                 'invalid', 'misleading', 'mistaken', 'phony',
      #                 'specious', 'spurious', 'unfounded', 'unreal',
      #                 'untrue',  'untruthful',  'apocryphal',  'beguiling',
      #                 'casuistic',  'concocted', 'cooked-up',
      #                 'counterfactual', 'deceiving', 'delusive', 'ersatz',
      #                 'fallacious', 'fishy',  'illusive',  'imaginary',
      #                 'inexact',  'lying',  'mendacious',
      #                 'misrepresentative', 'off the mark', 'sham',
      #                 'sophistical', 'trumped up', 'unsound']),
      # has_false_antonyms = set(['accurate', 'authentic', 'correct', 'fair', 'faithful',
      #                 'frank', 'genuine', 'honest', 'moral', 'open', 'proven',
      #                 'real', 'right', 'sincere', 'sound', 'true',
      #                 'trustworthy', 'truthful', 'valid', 'actual', 'factual',
      #                 'just', 'known', 'precise', 'reliable', 'straight',
      #                 'substantiated'])
)

In [ ]:
## FEATURE EXTRACTION ##

def extract_features(df):
    hasqmark = udf(lambda x: int('?' in x), IntegerType())
    df = df.withColumn('hasqmark', hasqmark(col('cleaned_text')))

    hasmark = udf(lambda x: int('!' in x), IntegerType())
    df = df.withColumn('hasmark', hasmark(col('cleaned_text')))

    hasperiod = udf(lambda x: int('.' in x), IntegerType())
    df = df.withColumn('hasperiod', hasperiod(col('cleaned_text')))

    df = df.withColumn('hashtags_count', expr('size(entities.hashtags)'))

    df = df.withColumn('mentions_count', expr('size(entities.user_mentions)'))

    df = df.withColumn('hasurls', expr('cast(size(entities.urls) >= 1 AS int)'))

    df = df.withColumn('hasmedia', expr('cast(size(entities.media) >= 1 AS int)'))

    df = df.withColumn('friends_count', expr('user.friends_count'))

    df = df.withColumn('followers_count', expr('user.followers_count'))

    ratiocapital = udf(lambda x: sum(map(str.isupper, x))/(len(x)+1), FloatType())
    df = df.withColumn('ratiocapital', ratiocapital(col('cleaned_text')))

    charlen = udf(lambda x: len(x), IntegerType())
    df = df.withColumn('charlen', charlen(col('cleaned_text')))

    df = df.withColumn('issource', expr('CAST((in_reply_to_status_id IS NULL) AS INT)'))

    ## TOKENIZATION ##

    tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
    temp_df = tokenizer.transform(df)

    #TODO remove stop words?
    #--eric--

    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    df = remover.transform(temp_df)

    hashingTF = HashingTF(inputCol="filtered", outputCol="rawhashtf", numFeatures=100)
    #hashingTF = HashingTF(inputCol="words", outputCol="rawhashtf", numFeatures=100)
    #--eric--
    df = hashingTF.transform(df)

    idf = IDF(inputCol="rawhashtf", outputCol="hashtf")
    idfModel = idf.fit(df)
    df = idfModel.transform(df)
    # TODO join with df so it has hashtf column

    ## TOKEN FEATURE EXTRACTION ##
    wordlen = udf(lambda words: len(words), IntegerType())
    df = df.withColumn('wordlen', wordlen(col('words')))

    def contains(y, x):
      return int(bool(len(y.intersection(set(x)))))


    for name, ys in words_dict.items():
        df = df.withColumn(name, udf(partial(contains, ys), IntegerType())(col('words')))



    #TODO negation words etc.
    negationwords = ['not', 'no', 'nobody', 'nothing', 'none', 'never',
              'neither', 'nor', 'nowhere', 'hardly', 'scarcely',
              'barely', 'don', 'isn', 'wasn', 'shouldn', 'wouldn',
              'couldn', 'doesn']
    def negacount(words):
      c = 0
      for negationword in negationwords:
        if negationword in words:
          c += 1
      return c
    negationcount = udf(negacount, IntegerType())
    df = df.withColumn('hasnegation', negationcount(col('words')))

    @udf('float')
    def count_upper(x):
        a = x.split()
        return sum(map(str.isupper, a))/(len(a) + 1)

    df = df.withColumn('allcapsratio', count_upper(col('cleaned_text')))
    #eric

    ## SENTENCE VECTORS  ##
    ## GLOVE ##
    # model = SentenceTransformer('average_word_embeddings_glove.6B.300d').eval()
    # temp = (df.select(['id','cleaned_text']).collect())

    # j = model.encode([x.cleaned_text for x in temp])

    # glove_rows = [Row(id=temp[i]['id'], glove=Vectors.dense(j[i])) for i in range(len(j))]

    # glove_schema = StructType([
    #   StructField("id", LongType(), True),
    #   StructField("glove_vector", VectorUDT(), True)
    # ])

    # d_df = spark.createDataFrame(glove_rows, glove_schema)

    # df  = df.join(d_df, on='id')

    ## BERT ##
    model = SentenceTransformer('stsb-distilbert-base').eval()

    temp = (df.select(['id','cleaned_text']).collect())

    j = model.encode([x.cleaned_text for x in temp], convert_to_tensor=True)

    bert_rows = [Row(id=temp[i]['id'], glove=Vectors.dense(j[i])) for i in range(len(j))]

    bert_schema = StructType([
      StructField("id", LongType(), True),
      StructField("bert_vector", VectorUDT(), True)
    ])

    d_df = spark.createDataFrame(bert_rows, bert_schema)

    df  = df.join(d_df, on='id')

    ### other models SentenceTransformer
    # model = SentenceTransformer('stsb-roberta-large').eval()

    # temp = (df.select(['id','cleaned_text']).collect())

    # j = model.encode([x.cleaned_text for x in temp])

    # bert_rows = [Row(id=temp[i]['id'], glove=Vectors.dense(j[i])) for i in range(len(j))]

    # bert_schema = StructType([
    #   StructField("id", LongType(), True),
    #   StructField("bert_vector_other_large", VectorUDT(), True)
    # ])

    # d_df = spark.createDataFrame(bert_rows, bert_schema)

    # df  = df.join(d_df, on='id')

    ### base
    # model = SentenceTransformer('stsb-roberta-base').eval()

    # temp = (df.select(['id','cleaned_text']).collect())

    # j = model.encode([x.cleaned_text for x in temp], convert_to_tensor=True)

    # bert_rows = [Row(id=temp[i]['id'], glove=Vectors.dense(j[i])) for i in range(len(j))]

    # bert_schema = StructType([
    #   StructField("id", LongType(), True),
    #   StructField("bert_vector_other_base", VectorUDT(), True)
    # ])

    # d_df = spark.createDataFrame(bert_rows, bert_schema)

    # df  = df.join(d_df, on='id')

    return df


In [ ]:
source_preprocessed = extract_features(clean(source_tweets_df))
reply_preprocessed = extract_features(clean(reply_tweets_df))

In [ ]:
print(source_preprocessed.count())
print(reply_preprocessed.count())

325
5243


In [ ]:
# temp = source_preprocessed.selectExpr('id AS in_reply_to_status_id', 'glove_vector AS glove_vector_source')
# reply_preprocessed = reply_preprocessed.join(temp, 'in_reply_to_status_id')

temp = source_preprocessed.selectExpr('id AS in_reply_to_status_id', 'bert_vector AS bert_vector_source')
reply_preprocessed = reply_preprocessed.join(temp, 'in_reply_to_status_id')

# temp = source_preprocessed.selectExpr('id AS in_reply_to_status_id', 'bert_vector_other_large AS bert_vector_other_large_source')
# reply_preprocessed = reply_preprocessed.join(temp, 'in_reply_to_status_id')

# temp = source_preprocessed.selectExpr('id AS in_reply_to_status_id', 'bert_vector_other_base AS bert_vector_other_base_source')
# reply_preprocessed = reply_preprocessed.join(temp, 'in_reply_to_status_id')


In [ ]:
# I think this is duplicated, arent we already removing mentions in the cleaning section?
#eric
#add cleaning special characters : '@'
# from pyspark.sql.functions import regexp_replace, col
# reply_preprocessed = reply_preprocessed.withColumn("cleaned_text",regexp_replace(col("cleaned_text"), "@", ""))


In [ ]:
# Flattening is done with the assembler in the pipeline below
#eric
#flatten the labels into one column
# reply_preprocessed_rdd = reply_preprocessed.select('cleaned_text').rdd
# reply_preprocessed_rdd.flatMap(lambda x: x).collect()

In [ ]:
# source_preprocessed = source_preprocessed.withColumn('glove_vector_source', udf(lambda : Vectors.zeros(300), VectorUDT())())
# source_preprocessed = source_preprocessed.withColumn('bert_vector_source', udf(lambda : Vectors.zeros(768), VectorUDT())())

In [ ]:
#eric
#cosine similarity bert_vector and original
# embed_bert = np.array([x for x in reply_preprocessed.select('bert_vector').collect()])
# nsamples, nx, ny = embed_bert.shape
# embed_bert = embed_bert.reshape((nsamples,nx*ny))
# embed_mat2 = np.array([x for x in reply_preprocessed.select('hashtf').collect()])
# nsamples, nx, ny = embed_mat2.shape
# embed_mat2 = embed_mat2.reshape((nsamples,nx*ny))
# sim_mat = cosine_similarity(embed_bert,embed_bert,embed_mat2)
#cosine similarity glove_vector and original
# embed_mat = np.array([x for x in reply_preprocessed.select('glove_vector').collect()])


In [ ]:
# TODO sim w/ source tweet
# TODO fix nans
@udf('float')
def cosine_udf(x,y):
  return max(0,float(1 - x.dot(y)/(x.norm(2)*y.norm(2))))

# reply_preprocessed = reply_preprocessed.withColumn('wrt_source_glove', cosine_udf(col('glove_vector'), col('glove_vector_source')))
reply_preprocessed = reply_preprocessed.withColumn('wrt_source_bert', cosine_udf(col('bert_vector'), col('bert_vector_source')))
# # reply_preprocessed = reply_preprocessed.withColumn('wrt_source_bert_large', util.pytorch_cos_sim(col('glove_vector'), col('bert_vector_other_large_source')))
# # reply_preprocessed = reply_preprocessed.withColumn('wrt_source_bert_base', util.pytorch_cos_sim(col('bert_vector'), col('bert_vector_other_base_source')))
# source_preprocessed = source_preprocessed.withColumn('wrt_source_glove', lit(0))
# source_preprocessed = reply_preprocessed.withColumn('wrt_source_bert', lit(0))

# TODO sim w/ thread

In [ ]:
# model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# # Two lists of sentences
# sentences1 = ['The cat sits outside',
#              'A man is playing guitar',
#              'The new movie is awesome']

# sentences2 = ['The dog plays in the garden',
#               'A woman watches TV',
#               'The new movie is so great']

# #Compute embedding for both lists
# embeddings1 = model.encode(sentences1, convert_to_tensor=True)
# embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# #Compute cosine-similarits
# cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

# #Output the pairs with their score
# for i in range(len(sentences1)):
#     print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

In [ ]:
# reply_preprocessed.na.fill(value=0,subset=["wrt_source_glove"])
# reply_preprocessed.select('ratiocapital').take(10)

In [ ]:
all_features = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls hasmedia
ratiocapital charlen issource wordlen hasnegation allcapsratio hashtf
favorite_count friends_count followers_count""".split() + list(words_dict.keys())
# reply_features = "glove_vector_source bert_vector_source wrt_source_glove wrt_source_bert wrt_source_bert_large wrt_source_bert_base".split()
# reply_features = "glove_vector_source bert_vector_source wrt_source_glove wrt_source_bert".split()
reply_features = []

In [ ]:
reply_preprocessed.select(['id'] + all_features + reply_features).union(
        source_preprocessed.select(['id'] + all_features + reply_features)).select('id').distinct().count()

3629

In [ ]:
train_tweets = reply_preprocessed.select(['id'] + all_features + reply_features).union(
        source_preprocessed.select(['id'] + all_features + reply_features)
    )
train_all = train_key_taskA.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join(train_tweets, 'id'
)

dev_tweets =   reply_preprocessed.select(['id'] + all_features + reply_features).union(
       source_preprocessed.select(['id'] + all_features + reply_features) 
)

dev = dev_key_taskA.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join( dev_tweets, 'id'
)

In [ ]:
print(train_key_taskA.count())
print(train_tweets.count())
print(train_all.count())
print(dev_key_taskA.count())
print(dev_tweets.count())
print(dev.count())

5217
3629
2899
1485


In [ ]:
train_test = train_key_taskA.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join(train_tweets, 'id', 'right')

In [ ]:
train_test.count()

3629

In [ ]:
# ML TEST
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [ ]:
# inputCols = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls
# hasmedia ratiocapital charlen issource wordlen hasnegation allcapsratio wrt_source_bert""".split()

inputCols = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls
hasmedia ratiocapital charlen issource wordlen hasnegation allcapsratio""".split() + list(words_dict.keys())

assembler = VectorAssembler(inputCols=inputCols,outputCol="features")
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)
indexer = StringIndexer(inputCol="label", outputCol="label_index")
pipeline = Pipeline(stages=[assembler, scaler, indexer])

processor = pipeline.fit(train_all)

temp = processor.transform(train_all)
train_all_features_df = temp.select(['features', 'label_index'])

temp = processor.transform(dev)
dev_features_df = temp.select(['features', 'label_index'])

In [ ]:
# layers = [
#           train_all_features_df.schema[0].metadata["ml_attr"]["num_attrs"],
#           128,
#           4
# ]
# trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234, labelCol='label_index')

# model = trainer.fit(train_all_features_df, )

In [ ]:
from pyspark.ml.classification import LogisticRegression
trainer = LogisticRegression(maxIter=10, regParam=0.001, labelCol='label_index')
model = trainer.fit(train_all_features_df, )

In [ ]:
# from pyspark.ml.classification import RandomForestClassifier
# trainer = RandomForestClassifier(numTrees=10, labelCol='label_index')
# model = trainer.fit(train_all_features_df, )

In [ ]:


# compute f1 on the dev set
result = model.transform(dev_features_df)
predictionAndLabels = result.select("prediction", "label_index")
evaluator = MulticlassClassificationEvaluator(metricName="f1", labelCol='label_index')


In [ ]:
# train_all_features_df.schema[0].metadata["ml_attr"]["num_attrs"]
print("Test set f1 = " + str(evaluator.evaluate(predictionAndLabels)))

Test set f1 = 0.7003150902561994


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = result.select(['label_index']).collect()
y_pred = result.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.76      0.96      0.85       500
         1.0       0.74      0.32      0.45        87
         2.0       0.71      0.52      0.60        81
         3.0       0.00      0.00      0.00        62

    accuracy                           0.75       730
   macro avg       0.55      0.45      0.47       730
weighted avg       0.69      0.75      0.70       730



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save('subtaskA_model')

In [ ]:
!zip -r subtaskA_model subtaskA_model/

  adding: subtaskA_model/ (stored 0%)
  adding: subtaskA_model/metadata/ (stored 0%)
  adding: subtaskA_model/metadata/._SUCCESS.crc (stored 0%)
  adding: subtaskA_model/metadata/part-00000 (deflated 44%)
  adding: subtaskA_model/metadata/.part-00000.crc (stored 0%)
  adding: subtaskA_model/metadata/_SUCCESS (stored 0%)
  adding: subtaskA_model/data/ (stored 0%)
  adding: subtaskA_model/data/._SUCCESS.crc (stored 0%)
  adding: subtaskA_model/data/part-00000-4f9020c3-b471-4e19-8a08-b82c8ba49617-c000.snappy.parquet (deflated 61%)
  adding: subtaskA_model/data/.part-00000-4f9020c3-b471-4e19-8a08-b82c8ba49617-c000.snappy.parquet.crc (stored 0%)
  adding: subtaskA_model/data/_SUCCESS (stored 0%)


In [ ]:
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
ExtractFeatureImp(model.featureImportances, dev_features_df, 'features')


AttributeError: ignored

In [ ]:
### ADD has_false_synonyms has_false_antonyms
all_features = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls hasmedia
ratiocapital charlen issource wordlen hasnegation allcapsratio hashtf
favorite_count friends_count followers_count""".split() + list(words_dict.keys())
# reply_features = "glove_vector_source bert_vector_source wrt_source_glove wrt_source_bert wrt_source_bert_large wrt_source_bert_base".split()
# reply_features = "glove_vector_source bert_vector_source wrt_source_glove wrt_source_bert".split()
reply_features = []

train_tweets = reply_preprocessed.select(['id'] + all_features + reply_features).union(
        source_preprocessed.select(['id'] + all_features + reply_features)
    )
train_all = train_key_taskA.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join(train_tweets, 'id'
)

dev_tweets =   reply_preprocessed.select(['id'] + all_features + reply_features).union(
       source_preprocessed.select(['id'] + all_features + reply_features) 
)

dev = dev_key_taskA.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join( dev_tweets, 'id'
)

# ML TEST
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

inputCols = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls
hasmedia ratiocapital charlen issource wordlen hasnegation allcapsratio""".split() + list(words_dict.keys())

assembler = VectorAssembler(inputCols=inputCols,outputCol="features")
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)
indexer = StringIndexer(inputCol="label", outputCol="label_index")
pipeline = Pipeline(stages=[assembler, scaler, indexer])

processor = pipeline.fit(train_all)

temp = processor.transform(train_all)
train_all_features_df = temp.select(['features', 'label_index'])

temp = processor.transform(dev)
dev_features_df = temp.select(['features', 'label_index'])

from pyspark.ml.classification import LogisticRegression
trainer = LogisticRegression(maxIter=10, regParam=0.001, labelCol='label_index')
model = trainer.fit(train_all_features_df, )


# compute f1 on the dev set
result = model.transform(dev_features_df)
predictionAndLabels = result.select("prediction", "label_index")
evaluator = MulticlassClassificationEvaluator(metricName="f1", labelCol='label_index')


In [ ]:
print("Test set f1 = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = result.select(['label_index']).collect()
y_pred = result.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

In [ ]:
indexer

In [ ]:
### TESTS ##
# df.select('hasurl').describe().show()
# temp_df.select('hastf').take(5)
# temp_df.select('words').take(5)
# df.select('hasnegation').filter(df.hasnegation > 0).take(5)
# df.select('allcapsratio').describe().show()
# df.select('ratiocapital').describe().show()
# df.selectExpr('CAST((in_reply_to_status_id IS NULL) AS INT)').describe().show()

# df.selectExpr('size(entities["media"])').describe().show()
# df.selectExpr('size(entities["urls"])').describe().show()
# df.selectExpr('size(entities["hashtags"])').describe().show()
# df.selectExpr('size(entities["symbols"])').describe().show()
# df.selectExpr('size(entities["urls"])').describe().show()
# df.selectExpr('size(entities["user_mentions"])').describe().show()
# df.printSchema()


In [ ]:
# from sparknlp.base import *
# from sparknlp.annotator import *
# from pyspark.ml import Pipeline
# documentAssembler = DocumentAssembler() \
#     .setInputCol("cleaned_text") \
#     .setOutputCol("document")

# regexTokenizer = Tokenizer() \
#     .setInputCols(["document"]) \
#     .setOutputCol("token")

# ner_dl = WordEmbeddings.pretrained('glove_100d')
# x_df = documentAssembler.transform(df)
# x_df = regexTokenizer.fit(x_df).transform(x_df)
# sentiment_detector = SentimentDetector.pretrained() \
#     .setInputCols(["document", "token"]) \
#     .setOutputCol("sentiment") \
#     .setLazyAnnotator(False)
# x_df = sentiment_detector.transform(x_df)
# x_df.select('sentiment').take(5)

In [ ]:
# reply_preprocessed.select('glove_vector').take(10)

In [ ]:
# reply_preprocessed.select('cleaned_text').take(10)

In [ ]:
# from pyspark.sql.functions import regexp_replace
# df = spark.createDataFrame([('@CBCNews',)], ['str'])
# reply_preprocessed.withColumn('cleaned_text',regexp_replace(col('cleaned_text'), r'(@([A-Za-z0-9]+))', '')).select('cleaned_text').take(10)


In [ ]:
# reply_preprocessed.groupby('in_reply_to_status_id').agg(collect_list('id').alias('replies_id')).take(10)

In [ ]:
# SentenceTransformer('average_word_embeddings_glove.6B.300d').encode('😱')

In [ ]:
# reply_preprocessed.groupby('in_reply_to_status_id').agg(collect_list('wrt_source_bert').alias('avg_cos')).take(10)

In [ ]:
all_features = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls hasmedia
ratiocapital charlen issource wordlen hasnegation allcapsratio hashtf
favorite_count friends_count followers_count""".split()

In [ ]:
def avg_cos(wrt_source_bert_list):
  n = len(wrt_source_bert_list)
  sum = 0
  for i in wrt_source_bert_list:
    sum += i
  return sum/n

avg_cos_udf = udf(avg_cos, FloatType())

# reply_preprocessed.groupby('in_reply_to_status_id').agg(collect_list('wrt_source_bert').alias('avg_cos')).withColumn('avg_cos', avg_cos_udf('avg_cos')).take(10)


In [ ]:
temp = reply_preprocessed.groupBy('in_reply_to_status_id').agg({'wrt_source_bert':'avg'})


In [ ]:
def rename_cols(agg_df, ignore_first_n=1):
    """changes the default spark aggregate names `avg(colname)` 
    to something a bit more useful. Pass an aggregated dataframe
    and the number of aggregation columns to ignore.
    """
    delimiters = "(", ")"
    split_pattern = '|'.join(map(re.escape, delimiters))
    splitter = partial(re.split, split_pattern)
    split_agg = lambda x: '_'.join(splitter(x))[0:-ignore_first_n]
    renamed = map(split_agg, agg_df.columns[ignore_first_n:])
    renamed = zip(agg_df.columns[ignore_first_n:], renamed)
    for old, new in renamed:
        agg_df = agg_df.withColumnRenamed(old, new)
    return agg_df
temp = rename_cols(temp)

In [ ]:
# temp = reply_preprocessed.groupby('in_reply_to_status_id').agg(collect_list('wrt_source_bert').alias('avg_cos')).withColumn('avg_cos', avg_cos_udf('avg_cos'))
source_all_features = source_preprocessed.join(temp, source_preprocessed.id == temp.in_reply_to_status_id)

In [ ]:
# 	idx	name	score
# 9	9	issource	0.282258
# 4	4	mentions_count	0.273590
# 1	1	hasqmark	0.165910
# 6	6	hasmedia	0.073971
# 5	5	hasurls	0.070193
# 8	8	charlen	0.031644
# 10	10	wordlen	0.022455
# 7	7	ratiocapital	0.019213
# 2	2	hasperiod	0.016388
# 3	3	hashtags_count	0.014601
# 12	12	allcapsratio	0.010897
# 19	19	has_question_words	0.006331
# 11	11	hasnegation	0.004701
# 16	16	has_knowledge	0.003167
# 0	0	hasmark	0.002603
# 14	14	has_report_words	0.001570
# 18	18	has_curse_words	0.000283
# 13	13	has_belief_words	0.000226
# 15	15	has_doubt_words	0.000000
# 17	17	has_denial_words	0.000000
# 20	20	has_other_words	0.000000

In [ ]:
### TASK B 
###
source_features = []

train_all_taskb = train_key_taskB.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join(
    source_all_features.select(['id'] + all_features + source_features), 'id'
)
dev_taskb = dev_key_taskB.withColumnRenamed('key', 'id').withColumnRenamed('value', 'label').join(
    source_all_features.select(['id'] + all_features + source_features), 'id'
)

In [ ]:
print(train_key_taskB.count())
print(train_all_taskb.count())
print(dev_key_taskB.count())
print(dev_taskb.count())

327
296
38
28


In [ ]:
# ML TEST
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

inputCols = """hasmark hasqmark hasperiod hashtags_count mentions_count hasurls
hasmedia ratiocapital charlen issource wordlen hasnegation allcapsratio hashtf
favorite_count friends_count followers_count""".split() + source_features

assembler = VectorAssembler(inputCols=inputCols,outputCol="features")
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)
indexer = StringIndexer(inputCol="label", outputCol="label_index")
pipeline = Pipeline(stages=[assembler, scaler, indexer])

processor = pipeline.fit(train_all_taskb)

temp = processor.transform(train_all_taskb)
train_all_taskb_features_df = temp.select(['features', 'label_index'])

temp = processor.transform(dev_taskb)
dev_taskb_features_df = temp.select(['features', 'label_index'])

In [ ]:
from pyspark.ml.classification import LogisticRegression
trainer = LogisticRegression(maxIter=10, regParam=0.001, labelCol='label_index')
model = trainer.fit(train_all_taskb_features_df, )

In [ ]:
result = model.transform(dev_taskb_features_df)
predictionAndLabels = result.select("prediction", "label_index")
evaluator = MulticlassClassificationEvaluator(metricName="f1", labelCol='label_index')

In [ ]:
print("Test set f1 = " + str(evaluator.evaluate(predictionAndLabels)))

Test set f1 = 0.3277504105090312


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = result.select(['label_index']).collect()
y_pred = result.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.29      0.75      0.41         8
         1.0       0.50      0.25      0.33         8
         2.0       0.67      0.17      0.27        12

    accuracy                           0.36        28
   macro avg       0.48      0.39      0.34        28
weighted avg       0.51      0.36      0.33        28

